In [24]:
! pip install -qU pandas azure-storage-blob pypdf tabula-py langchain-openai langchain-community faiss-cpu langchain-google-genai

In [54]:
import dotenv
dotenv.load_dotenv()

True

In [16]:
import os
import io
import json
import pandas as pd
import PyPDF2
import time
import tabula
from tabula.io import read_pdf
from azure.storage.blob import ContainerClient
from langchain_openai import AzureChatOpenAI

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
# from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory


In [3]:
# Set up Azure Blob Storage connection
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
container_name = os.getenv("CONTAINER_NAME")  # Replace with your container name
container_client = ContainerClient.from_connection_string(
    conn_str=connection_string,
    container_name=container_name
)

In [4]:
import tabula.io


def extract_tables(path):
    # path = "q1-2017-press-release.pdf"

    tables = tabula.io.read_pdf(path, pages='all', multiple_tables=True)

    cleaned_tables = []

    for i, table in enumerate(tables):
        if table.empty:
        #   print(f"Table {i+1} is empty or poorly formatted and will skiped")
          continue

        table.dropna(how='all',inplace=True)

        table.dropna(how='all', axis=1, inplace=True)

        table.columns = table.columns.str.strip()

        # table_json =table.to_json(orient = 'records')
        # with open(f"table_{i+1}.json", "w") as json_file:
        #     json_file.write(table_json)

        # print(f"Cleaned Table {i+1} extracted and saved as table_{i+1}.json")

        cleaned_tables.append(table)

        # print(f"\nCleaned Table {i+1}:\n", table)
    # print(f"\n Total tables extracted : {len(cleaned_tables)}")
    return cleaned_tables

In [5]:
documents = []
company_name = "Alphabet"
blobs_stream = []
extracted_tables = []
for blob in container_client.list_blob_names():
    if company_name in blob:
        stream  = io.BytesIO()
        container_client.download_blob(blob).readinto(stream)
        temp_file = "temp.pdf"
        with open(temp_file, "wb") as file:
            file.write(stream.getvalue())
        blobs_stream.append(stream)
        loader = PyPDFLoader(temp_file)
        docs = loader.load()
        documents.extend(docs)

In [6]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("ENDPOINT_URL")
)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
splits = text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [56]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     api_key = os.getenv("GOOGLE_API_KEY")
#     # other params...
# )
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("DEPLOYMENT_NAME"),  # or your deployment
    api_version="2023-06-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("ENDPOINT_URL")
    # other params...
)
llm.invoke('who are you?')

AIMessage(content="I'm an artificial intelligence developed by OpenAI, known as ChatGPT. I'm here to help answer your questions, provide information, and assist with a variety of tasks. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 11, 'total_tokens': 51, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-dfbbf30e-3d3b-4452-a54c-09f546bad92e-0', usage_metadata={'input_tokens': 11, 'output_tokens': 40, 'total_tokens': 51})

In [57]:
contextualize_q_system_prompt=(
    "Given a chat history and the latest user question"
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

In [58]:
system_prompt = (
    """
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, simply say "Data Not Available" 
    Do not answer in sentences, try to answer in one word.
    \n\n
    ```context
    {context}
    ```
    """
)
qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

In [59]:
store = {}
# session_id = "abc"
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
conversational_rag_chain=RunnableWithMessageHistory(
    rag_chain,get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [60]:
output = {}
for year in range(2014, 2025):
    for q in range(1, 5):
        session_id = f"abc_q{q} {year}"
        user_input = f"""what is the value of capital expenditure in q{q} {year}."""
        session_history=get_session_history(session_id)
        response = conversational_rag_chain.invoke(
            {"input": user_input},
            config={
                "configurable": {"session_id":session_id}
            },  # constructs a key "abc123" in `store`.
        )
        output[f"q{q}_{year}"] = response['answer']

In [61]:
output

{'q1_2014': 'Data Not Available',
 'q2_2014': 'Data Not Available',
 'q3_2014': 'Data Not Available',
 'q4_2014': 'Data Not Available',
 'q1_2015': 'Data Not Available',
 'q2_2015': 'Data Not Available',
 'q3_2015': 'Data Not Available',
 'q4_2015': 'Data Not Available',
 'q1_2016': 'Data Not Available',
 'q2_2016': 'Data Not Available',
 'q3_2016': 'Data Not Available',
 'q4_2016': 'Data Not Available',
 'q1_2017': 'Data Not Available',
 'q2_2017': 'Data Not Available',
 'q3_2017': 'Data Not Available',
 'q4_2017': 'Data Not Available',
 'q1_2018': 'Data Not Available',
 'q2_2018': 'Data Not Available',
 'q3_2018': 'Data Not Available',
 'q4_2018': 'Data Not Available',
 'q1_2019': 'Data Not Available',
 'q2_2019': 'Data Not Available',
 'q3_2019': 'Data Not Available',
 'q4_2019': 'Data Not Available',
 'q1_2020': '$6,005 million',
 'q2_2020': '$5,391 million',
 'q3_2020': '$5,406 million',
 'q4_2020': '$5,479 million',
 'q1_2021': '$5,942 million',
 'q2_2021': '$6,828 million',
 'q3

In [41]:
user_input = """Provide an analysis of changes in capital expenditure (CapEx) organized by quarter (e.g. q1 2023, q2 2023, q3 2023 etc.).
The response of each quarter should fall into one of the following categories: Increase, Decrease, No change, or Data Unavailable."""
session_history=get_session_history(session_id)
response = conversational_rag_chain.invoke(
    {"input": user_input},
    config={
        "configurable": {"session_id":session_id}
    },  # constructs a key "abc123" in `store`.
)
print(f"{'*'*5}HISTORY{'*'*5}\n{session_history.messages}{'*'*5}{'*'*5}")
response['answer']

*****HISTORY*****
[HumanMessage(content='Provide an analysis of changes in capital expenditure (CapEx) organized by quarter (e.g. q1 2023, q2 2023, q3 2023 etc.).\nThe response of each quarter should fall into one of the following categories: Increase, Decrease, No change, or Data Unavailable.', additional_kwargs={}, response_metadata={}), AIMessage(content='Data on capital expenditure for all quarters besides Q4 2022 is unavailable in the provided context. \n\n**Q4 2022:**  $7.595 billion \n\n**Other Quarters:** Data Unavailable \n', additional_kwargs={}, response_metadata={})]**********


'Data on capital expenditure for all quarters besides Q4 2022 is unavailable in the provided context. \n\n**Q4 2022:**  $7.595 billion \n\n**Other Quarters:** Data Unavailable \n'